<a href="https://colab.research.google.com/github/MichaelBieri/Sequence-Forecast-Energy-Values-using-Multivariate-Analysis/blob/main/Tarifvergleich2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RENOLD & PARTNER GMBH ANFRAGE ENERGIE**
Mai 2025

*https://github.com/zazuko/notebooks/blob/master/notebooks/electricity_prices/electricity_prices.ipynb*

In [ ]:
import json
import re
import string

import folium
import mapclassify
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from graphly.api_client import SparqlClient

%matplotlib inline

In [ ]:
# Uncomment to install dependencies in Colab environment
!pip install mapclassify
!pip install git+https://github.com/zazuko/graphly.git

In [ ]:
sparql = SparqlClient("https://lindas.admin.ch/query")
geosparql = SparqlClient("https://geo.ld.admin.ch/query")

sparql.add_prefixes({
    "schema": "<http://schema.org/>",
    "cube": "<https://cube.link/>",
    "elcom": "<https://energy.ld.admin.ch/elcom/electricityprice/dimension/>",
    "admin": "<https://schema.ld.admin.ch/>"
})

geosparql.add_prefixes({
    "dct": "<http://purl.org/dc/terms/>",
    "geonames": "<http://www.geonames.org/ontology#>",
    "schema": "<http://schema.org/>",
    "geosparql": "<http://www.opengis.net/ont/geosparql#>",
})

Anfrage: Total electricity price for municipalities

In [ ]:
query = """
SELECT DISTINCT ?postalCode ?total ?operatorName
FROM <https://lindas.admin.ch/elcom/electricityprice>
WHERE {

  ?offer a schema:Offer ;
    schema:areaServed ?muniri ;
    schema:postalCode ?postalCode .

  <https://energy.ld.admin.ch/elcom/electricityprice> cube:observationSet/cube:observation ?obs .

  ?obs a cube:Observation ;
    elcom:period "2023"^^xsd:gYear ; # data available up to 10 years ago
    elcom:municipality ?muniri ;
    elcom:category <https://energy.ld.admin.ch/elcom/electricityprice/category/H1> ; # categories are explained below
    elcom:operator ?operator ;
    elcom:product <https://energy.ld.admin.ch/elcom/electricityprice/product/standard> ; # Some municipalities also have a "cheapest" product
    elcom:total ?total .

  ?operator schema:name ?operatorName .

  # FILTER( ?postalCode = "2502")

}
"""

totalByPostCode = sparql.send_query(query)
totalByPostCode.head()